# Addition of stochastic PV production at busses 3 and 4
Extend with actual stochastic data

In [34]:
using JuMP, GLPK, HiGHS

# ------------------------------
# Data and Parameters
# ------------------------------

# Time periods
T = 1:4

# Scenarios
scenarios = 1:3

# Scenario probabilities (equal probabilities)
prob = Dict(1 => 1/3, 2 => 1/3, 3 => 1/3)

# Wind availability at Bus 5 (stochastic, for each scenario and time period)
W = Dict((1,1) => 70.0, (1,2) => 60.0, (1,3) => 80.0, (1,4) => 70.0,
         (2,1) => 50.0,  (2,2) => 40.0, (2,3) => 40.0, (2,4) => 60.0,
         (3,1) => 0.0,   (3,2) => 0.0,  (3,3) => 0.0,  (3,4) => 0.0)

# Dynamic load at Bus 2 (stochastic, for each scenario and time period)
D2 = Dict((1,1) => 10.0, (1,2) => 15.0, (1,3) => 20.0, (1,4) => 15.0,
          (2,1) => 10.0, (2,2) => 15.0, (2,3) => 11.0, (2,4) => 11.0,
          (3,1) => 12.0, (3,2) => 12.0, (3,3) => 13.0, (3,4) => 13.0)

# Stochastic PV availability at Bus 3 (maximum available, per scenario and time period)
PV3 = Dict((1,1) => 40.0, (1,2) => 35.0, (1,3) => 30.0, (1,4) => 20.0,
           (2,1) => 50.0, (2,2) => 45.0, (2,3) => 40.0, (2,4) => 30.0,
           (3,1) => 0.0,  (3,2) => 0.0,  (3,3) => 0.0,  (3,4) => 0.0)

# Stochastic PV availability at Bus 4 (maximum available, per scenario and time period)
PV4 = Dict((1,1) => 30.0, (1,2) => 25.0, (1,3) => 20.0, (1,4) => 15.0,
           (2,1) => 35.0, (2,2) => 30.0, (2,3) => 25.0, (2,4) => 20.0,
           (3,1) => 0.0,  (3,2) => 0.0,  (3,3) => 0.0,  (3,4) => 0.0)


# Fixed (ordinary) demands at buses (assumed constant over time)
d1 = 10.0   # Bus 1
d3 = 20.0   # Bus 3
d4 = 20.0   # Bus 4

# Conventional generation at Bus 1
Gmax = 300.0        # Maximum generation capacity
c_g = 10.0          # Generation cost per unit

# curtailment penalty for renewables
c_curt = 0.0

# Ramping limits for conventional generator (per time period)
ramp_up = 50.0
ramp_down = 50.0

# Network parameters for DC load flow
X51 = 1.0   # Bus 5 - Bus 1
X12 = 1.0   # Bus 1 - Bus 2
X23 = 1.0   # Bus 2 - Bus 3
X24 = 1.0   # Bus 2 - Bus 4
Fmax = 100.0  # Flow limits on each line


100.0

In [35]:
# ------------------------------
# Model Definition
# ------------------------------

model = Model(HiGHS.Optimizer)

# Conventional generation at Bus 1 (indexed by scenario and time)
@variable(model, g_conv[s in scenarios, t in T] >= 0, upper_bound = Gmax)

# Wind generation at Bus 5 (indexed by scenario and time)
@variable(model, g_wind[s in scenarios, t in T] >= 0)
@variable(model, c_w[s in scenarios, t in T] >= 0)

# No separate curtailment variable is needed; wind curtailment = (W - g_wind)

# PV generation at Bus 3 and Bus 4 (indexed by scenario and time)
@variable(model, g_PV3[s in scenarios, t in T] >= 0)
@variable(model, c_pv3[s in scenarios, t in T] >= 0)
@variable(model, g_PV4[s in scenarios, t in T] >= 0)
@variable(model, c_pv4[s in scenarios, t in T] >= 0)

# Network flows on each line (indexed by scenario and time)
@variable(model, f51[s in scenarios, t in T])
@variable(model, f12[s in scenarios, t in T])
@variable(model, f23[s in scenarios, t in T])
@variable(model, f24[s in scenarios, t in T])

# Voltage angles at buses 1 through 5 (indexed by scenario and time)
@variable(model, theta[bus in 1:5, s in scenarios, t in T])

# Fix reference angle at Bus 1 for all scenarios and time periods
for s in scenarios, t in T
    @constraint(model, theta[1,s,t] == 0)
end

# ------------------------------
# Objective Function
# ------------------------------

# The objective minimizes the cost of conventional generation and penalizes unused renewables
@objective(model, Min, 
    sum(prob[s] * (c_g * g_conv[s,t] + c_curt * (c_w[s,t] + c_pv3[s,t] + c_pv4[s,t])) for s in scenarios, t in T))

# ------------------------------
# Constraints
# ------------------------------

# Wind generation constraint: wind used must not exceed available wind.
for s in scenarios, t in T
    @constraint(model, g_wind[s,t] + c_w[s,t] == W[(s,t)])
end

# PV generation constraints at Bus 3 and Bus 4:
for s in scenarios, t in T
    @constraint(model, g_PV3[s,t] + c_pv3[s,t] == PV3[(s,t)])
    @constraint(model, g_PV4[s,t] + c_pv4[s,t] == PV4[(s,t)])
end

# DC power flow equations and flow limits for each line, scenario, and time period:
for s in scenarios, t in T
    @constraint(model, f51[s,t] == (theta[5,s,t] - theta[1,s,t]) / X51)
    @constraint(model, f12[s,t] == (theta[1,s,t] - theta[2,s,t]) / X12)
    @constraint(model, f23[s,t] == (theta[2,s,t] - theta[3,s,t]) / X23)
    @constraint(model, f24[s,t] == (theta[2,s,t] - theta[4,s,t]) / X24)
    
    @constraint(model, f51[s,t] <= Fmax)
    @constraint(model, -f51[s,t] <= Fmax)
    @constraint(model, f12[s,t] <= Fmax)
    @constraint(model, -f12[s,t] <= Fmax)
    @constraint(model, f23[s,t] <= Fmax)
    @constraint(model, -f23[s,t] <= Fmax)
    @constraint(model, f24[s,t] <= Fmax)
    @constraint(model, -f24[s,t] <= Fmax)
end

# Power balance constraints at each bus for each scenario and time period:
for s in scenarios, t in T
    # Bus 5 (wind bus): Wind generation flows to Bus 1.
    @constraint(model, g_wind[s,t] - f51[s,t] == 0)
    
    # Bus 1: Conventional generation plus inflow from Bus 5 meets its own demand and supplies Bus 2.
    @constraint(model, g_conv[s,t] - d1 + f51[s,t] - f12[s,t] == 0)
    
    # Bus 2: Receives power from Bus 1, meets its dynamic load, and sends power to Buses 3 and 4.
    @constraint(model, f12[s,t] - D2[(s,t)] - f23[s,t] - f24[s,t] == 0)
    
    # Bus 3: Local PV generation at Bus 3 now contributes to meeting the fixed demand.
    @constraint(model, f23[s,t] + g_PV3[s,t] - d3 == 0)
    
    # Bus 4: Local PV generation at Bus 4 now contributes to meeting the fixed demand.
    @constraint(model, f24[s,t] + g_PV4[s,t] - d4 == 0)
end

# Ramping constraints for conventional generation at Bus 1 (for each scenario, between consecutive time periods):
for s in scenarios, t in T[2:end]
    @constraint(model, g_conv[s,t] - g_conv[s,t-1] <= ramp_up)
    @constraint(model, g_conv[s,t-1] - g_conv[s,t] <= ramp_down)
end

In [36]:
# ------------------------------
# Solve the Model
# ------------------------------

optimize!(model)

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [3e+00, 3e+00]
  Bound  [3e+02, 3e+02]
  RHS    [1e+01, 1e+02]
Presolving model
42 rows, 48 cols, 96 nonzeros  0s
20 rows, 32 cols, 56 nonzeros  0s
6 rows, 8 cols, 12 nonzeros  0s
6 rows, 8 cols, 12 nonzeros  0s
Presolve : Reductions: rows 6(-264); columns 8(-184); elements 12(-492)
Solving the presolved LP
Solving the original LP from the solution after postsolve
Model status        : Optimal
Objective value     :  8.3333333333e+02
Relative P-D gap    :  1.3642420527e-16
HiGHS run time      :          0.00


In [37]:
# ------------------------------
# Output the Results
# ------------------------------

println("Overall Optimal Objective Value: ", objective_value(model))

# Print results per scenario and time period
for s in scenarios, t in T
    println("Scenario $s, Time Period $t:")
    println("  Conventional Generation (Bus 1): ", value(g_conv[s,t]))
    println("  Wind Generation Used (Bus 5): ", value(g_wind[s,t]))
    println("  Wind Curtailment (Bus 5): ", W[(s,t)] - value(g_wind[s,t]))
    println("  PV Generation at Bus 3: ", value(g_PV3[s,t]))
    println("  PV Generation at Bus 4: ", value(g_PV4[s,t]))
end

Overall Optimal Objective Value: 833.3333333333333
Scenario 1, Time Period 1:
  Conventional Generation (Bus 1): 0.0
  Wind Generation Used (Bus 5): 60.0
  Wind Curtailment (Bus 5): 10.0
  PV Generation at Bus 3: 0.0
  PV Generation at Bus 4: 0.0
Scenario 1, Time Period 2:
  Conventional Generation (Bus 1): 0.0
  Wind Generation Used (Bus 5): 5.0
  Wind Curtailment (Bus 5): 55.0
  PV Generation at Bus 3: 35.0
  PV Generation at Bus 4: 25.0
Scenario 1, Time Period 3:
  Conventional Generation (Bus 1): 0.0
  Wind Generation Used (Bus 5): 70.0
  Wind Curtailment (Bus 5): 10.0
  PV Generation at Bus 3: 0.0
  PV Generation at Bus 4: 0.0
Scenario 1, Time Period 4:
  Conventional Generation (Bus 1): 0.0
  Wind Generation Used (Bus 5): 65.0
  Wind Curtailment (Bus 5): 5.0
  PV Generation at Bus 3: 0.0
  PV Generation at Bus 4: 0.0
Scenario 2, Time Period 1:
  Conventional Generation (Bus 1): 0.0
  Wind Generation Used (Bus 5): 0.0
  Wind Curtailment (Bus 5): 50.0
  PV Generation at Bus 3: 25.0